##### Raw Alpha Vantage Api (https://www.alphavantage.co/documentation/)

In [1]:
import requests
import json
from io import StringIO
import pandas as pd

def save_quarterly_data(symbol, returning=False):
    alpha_key = ''
    with open('keys.json', 'r') as keys:
        credentials = json.load(keys)
        alpha_key = credentials.get('alphavantage')
    
    if not alpha_key: return('You need an api key for Alpha Vantage')
    
    values = get_data(symbol, alpha_key)
    if values is None:
        print(f'{symbol} not saved !!')
        return None
    
    income = requests.get(f'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol={symbol}&outputsize=full&apikey={alpha_key}')
    if 'quarterlyReports' not in json.loads(income.text):
        print(f'{symbol} not saved !!')
        return None
    income = pd.read_json(StringIO(json.dumps(json.loads(income.text)['quarterlyReports'])))
    
    if 'fiscalDateEnding' not in income.columns:
        print(f'{symbol} not saved !!')
        return None
    
    balance = requests.get(f'https://www.alphavantage.co/query?function=BALANCE_SHEET&symbol={symbol}&outputsize=full&apikey={alpha_key}')
    if 'quarterlyReports' not in json.loads(balance.text):
        print(f'{symbol} not saved !!')
        return None
    balance = pd.read_json(StringIO(json.dumps(json.loads(balance.text)['quarterlyReports'])))
    
    if 'fiscalDateEnding' not in balance.columns:
        print(f'{symbol} not saved !!')
        return None
    
    data = pd.merge(income, balance, on='fiscalDateEnding')
    
    cashflow = requests.get(f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={symbol}&outputsize=full&apikey={alpha_key}')
    if 'quarterlyReports' not in json.loads(cashflow.text):
        print(f'{symbol} not saved !!')
        return None
    cashflow = pd.read_json(StringIO(json.dumps(json.loads(cashflow.text)['quarterlyReports'])))
    
    if 'fiscalDateEnding' not in cashflow.columns:
        print(f'{symbol} not saved !!')
        return None
    
    data = pd.merge(data, cashflow, on='fiscalDateEnding')
    
    data = last_clossing(values, data)
    data = append_y(data)
    
    try: data.to_csv(f'./datasets/fundamental_data/{symbol}.csv')
    except: print(f'{symbol} not saved !!')
    
    if returning: return data

In [2]:
def get_data(symbol, alpha_key):
    csv = requests.get(f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize=full&apikey={alpha_key}&datatype=csv')
    data = pd.read_csv(StringIO(csv.text))
    
    if 'timestamp' not in data.columns or 'close' not in data.columns:
        return None
    
    return data[['timestamp', 'close']]

In [3]:
def last_clossing(values, data):
    for f in data['fiscalDateEnding']:
        g = f
        clossingVal = 'nan'
        while clossingVal == 'nan':
            try: clossingVal = values.loc[values['timestamp'] == g, 'close'].values[0]
            except: g = g[:-2]+str(int(g[-2:])-1)
                
            if int(f[-2:])-int(g[-2:]) > 2: break
        
        data.loc[data['fiscalDateEnding'] == f, 'clossingVal'] = clossingVal
        
    return data

In [4]:
def append_y(data):
    future = None
    for f in data['fiscalDateEnding']:
        if future is not None: data.loc[data['fiscalDateEnding'] == f, 'nextClossingVal'] = future
        future = data.loc[data['fiscalDateEnding'] == f, 'clossingVal'].values[0]
        
    return data

In [5]:
symbols = pd.read_csv('./datasets/nasdaq_list.csv', sep='|')
symbols = symbols[['Symbol']].to_numpy().reshape((len(symbols))).tolist()

In [ ]:
import time

for symbol in symbols[1612:]:
    save_quarterly_data(symbol)
    time.sleep(30)

HWBK not saved !!
HWCPL not saved !!
HX not saved !!
HYACU not saved !!
HYLS not saved !!
HYRE not saved !!
HYXE not saved !!
HYZD not saved !!
IART not saved !!
IBB not saved !!
IBBJ not saved !!
IBKR not saved !!
IBTA not saved !!
IBTB not saved !!
IBTD not saved !!
IBTE not saved !!
IBTF not saved !!
IBTG not saved !!
IBTH not saved !!
IBTI not saved !!
IBTJ not saved !!
IBTK not saved !!
IBTX not saved !!
IBUY not saved !!
ICCH not saved !!
ICLK not saved !!
ICLN not saved !!
ICLR not saved !!
ICPT not saved !!
IDEX not saved !!
IDLB not saved !!
IDN not saved !!
IDXX not saved !!
IEAWW not saved !!
IEF not saved !!
IEI not saved !!
IEUS not saved !!
IFGL not saved !!
IFMK not saved !!
IFV not saved !!
IGF not saved !!
IGIB not saved !!
IGIC not saved !!
IGMS not saved !!
IGOV not saved !!
IGSB not saved !!
IIIN not saved !!
IIVI not saved !!
IIVIP not saved !!
IJT not saved !!
ILPT not saved !!
IMAC not saved !!
IMGN not saved !!
IMMR not saved !!
IMRA not saved !!
IMRNW not saved